In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from JAxtar.hash import dataclass_hashing, dataclass_hashing_batch
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic
from JAxtar.hash import HashTable

In [3]:
count = int(1e5)
puzzle = SlidePuzzle(4)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(0),count))
table = HashTable.make_lookup_table(puzzle.State, 1, int(1e6))

idx, table_idx, found = jax.vmap(HashTable.check, in_axes=(None,0))(table, sample)
#print(idx, table_idx, found)

In [4]:
import time
from tqdm.autonotebook import trange
start = time.time()
batch = 1000
for i in trange(count // batch):
    table, _ = HashTable.batch_insert(table, sample[i*batch:(i+1)*batch])
print("insert time:", time.time()-start)
start = time.time()
idx, table_idx, found = jax.vmap(HashTable.check, in_axes=(None,0))(table, sample)
print("check time:", time.time()-start)
print(idx, table_idx, found)
print(jnp.sum(found) / count)

/tmp/ipykernel_18854/1518994947.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
100%|██████████| 100/100 [00:19<00:00,  5.20it/s]


insert time: 19.23161554336548
check time: 0.19525980949401855
[  1020 949507  57409 ... 715924 273794 200985] [0 0 0 ... 0 0 1] [ True  True  True ...  True  True  True]
1.0
